# First fermionic DMRG

Created 03/01/2025

Objectives:
* Construct the 1D fermionic spt states from the Wang & Gu paper, first pass. Work out the case with trivial projective representation first.
* Not sure of anyway to test the states...? Apart from running the SPT classification on them.

# Imports

In [1]:
import numpy as np
import scipy
import matplotlib.pyplot as plt

In [2]:
from itertools import product

In [3]:
import tenpy
import tenpy.linalg.np_conserved as npc
from tenpy.algorithms import dmrg
from tenpy.networks.mps import MPS

In [4]:
from tenpy.networks.site import ClockSite, FermionSite
from tenpy.models.lattice import Chain
from tenpy.models.model import CouplingModel, NearestNeighborModel, MPOModel, CouplingMPOModel

In [5]:
def mod_4_to_bit(x):    
    possible_mod_4_values = list(range(4))
    assert x in possible_mod_4_values

    x0 = x % 2
    x1 = ((x - x0) // 2) % 2

    return (x0, x1)

In [6]:
def bit_to_mod_4(bit_pair):
    x0, x1 = bit_pair

    possible_mod_2_values = list(range(2))
    assert x0 in possible_mod_2_values
    assert x1 in possible_mod_2_values

    return x0 + 2*x1

In [7]:
def mod_4_to_bit_addition(x, y):
    (x0, x1) = mod_4_to_bit(x)
    (y0, y1) = mod_4_to_bit(y)

    out0 = (x0 + y0) % 2
    out1 = (x1 + y1) % 2

    out = bit_to_mod_4((out0, out1))

    return out

In [8]:
np.array([
    [
        mod_4_to_bit_addition(x, y) for x in range(4)
    ]
    for y in range(4)
])

array([[0, 1, 2, 3],
       [1, 0, 3, 2],
       [2, 3, 0, 1],
       [3, 2, 1, 0]])

In [9]:
def get_n1_func(n1_01, n1_10):
    l = [0, n1_01, n1_10, (n1_01 + n1_10)%2]
    
    def f(g1, g2):
        i = mod_4_to_bit_addition(g1, g2)
        return l[i]

    return f

In [10]:
n1_pair_to_fermion_operators_dict = {
    (0, 0): "Id",
    (0, 1): "C",
    (1, 0): "Cd",
    (1, 1): "N"
}

In [11]:
def get_op_list(g_quad, n1_func):
    out = list()
    g_left, g_in, g_out, g_right = g_quad

    out.append((f'map_{g_left}_{g_left}', 0, 0))

    left_fermion_op_string = n1_pair_to_fermion_operators_dict[
        (n1_func(g_left, g_in), n1_func(g_left, g_out))
    ]

    out.append((left_fermion_op_string, 0, 1))
    
    out.append((f'map_{g_in}_{g_out}', 1, 0))

    right_fermion_op_string = n1_pair_to_fermion_operators_dict[
        (n1_func(g_in, g_right), n1_func(g_out, g_right))
    ]

    out.append((right_fermion_op_string, 1, 1))
    
    out.append((f'map_{g_right}_{g_right}', 2, 0))

    return out

In [12]:
class ClusterIsingFermion(CouplingMPOModel):
        default_lattice = "Chain"
        force_default_lattice = True

        # Would it be easier to subclass ClockSite?
        def init_sites(self, model_params):
            spin = ClockSite(4, conserve=None)
            for i in range(4):
                for j in range(4):
                    X = np.zeros((4,4))
                    X[j, i] = 1
                    op_name = f"map_{i}_{j}"
                    spin.add_op(op_name, X)
            ferm = FermionSite(conserve=None)
            sites = [spin, ferm]
            return [spin, ferm], ['s', 'f']

        def init_terms(self, model_params):
            # Read off model parameters
            n1_01, n1_10 = model_params.get('n1', (0,0))
            n1_func = get_n1_func(n1_01, n1_10)
            
            group_quads = product(range(4), repeat=4)

            for group_quad in group_quads:
                op_list = get_op_list(group_quad, n1_func)
                self.add_multi_coupling(-1/4, op_list)

In [13]:
M = ClusterIsingFermion({'L': 20})

In [13]:
n1_pairs = [
    (0, 0),
    (0, 1),
    (1, 0),
    (1, 1)
]

In [15]:
models = [
    ClusterIsingFermion({'L': 20, 'n1': p}) for p in n1_pairs
]

In [16]:
type(models[0])

__main__.ClusterIsingFermion

In [17]:
models

In [18]:
mpo = models[0].calc_H_MPO()

In [19]:
mpo.get_W(0)

<npc.Array shape=(2, 6, 4, 4) labels=['wL', 'wR', 'p', 'p*']>

In [20]:
mpo.get_W(0).to_ndarray()

array([[[[1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 1., 0.],
         [0., 0., 0., 1.]],

        [[1., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 1., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 1.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]],


       [[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        

In [21]:
mpo.get_W(1)

<npc.Array shape=(6, 6, 2, 2) labels=['wL', 'wR', 'p', 'p*']>

In [22]:
mpo.get_W(1).to_ndarray()

array([[[[1., 0.],
         [0., 1.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[1., 0.],
         [0., 1.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[1., 0.],
         [0., 1.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[1., 0.],
         [0., 1.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0.,

In [23]:
mpo.get_W(2).to_ndarray()

array([[[[ 1.  ,  0.  ,  0.  ,  0.  ],
         [ 0.  ,  1.  ,  0.  ,  0.  ],
         [ 0.  ,  0.  ,  1.  ,  0.  ],
         [ 0.  ,  0.  ,  0.  ,  1.  ]],

        [[ 1.  ,  0.  ,  0.  ,  0.  ],
         [ 0.  ,  0.  ,  0.  ,  0.  ],
         [ 0.  ,  0.  ,  0.  ,  0.  ],
         [ 0.  ,  0.  ,  0.  ,  0.  ]],

        [[ 0.  ,  0.  ,  0.  ,  0.  ],
         [ 0.  ,  1.  ,  0.  ,  0.  ],
         [ 0.  ,  0.  ,  0.  ,  0.  ],
         [ 0.  ,  0.  ,  0.  ,  0.  ]],

        [[ 0.  ,  0.  ,  0.  ,  0.  ],
         [ 0.  ,  0.  ,  0.  ,  0.  ],
         [ 0.  ,  0.  ,  1.  ,  0.  ],
         [ 0.  ,  0.  ,  0.  ,  0.  ]],

        [[ 0.  ,  0.  ,  0.  ,  0.  ],
         [ 0.  ,  0.  ,  0.  ,  0.  ],
         [ 0.  ,  0.  ,  0.  ,  0.  ],
         [ 0.  ,  0.  ,  0.  ,  1.  ]],

        [[ 0.  ,  0.  ,  0.  ,  0.  ],
         [ 0.  ,  0.  ,  0.  ,  0.  ],
         [ 0.  ,  0.  ,  0.  ,  0.  ],
         [ 0.  ,  0.  ,  0.  ,  0.  ]],

        [[ 0.  ,  0.  ,  0.  ,  0.  ],
         [ 0.

In [15]:
psis = [
    MPS.from_lat_product_state(m.lat, [['0', 'empty'],]*20)
    for m in models
]

In [16]:
dmrg_params = {
    "trunc_params": {"chi_max": 8, "chi_min": 1, "svd_min": 1.e-10},
    "min_sweeps":100,
    "max_sweeps":200,
    "mixer": True,
    "combine":False,
    'decay':2,
    'amplitude':10e-1,
    'disable_after':60,
    'update_env':0
}

In [17]:
for psi in psis:
    psi.canonical_form()

In [18]:
outpsis = list()
energies = list()

for psi, m in zip(psis, models):
    eng = dmrg.TwoSiteDMRGEngine(psi, m, dmrg_params)
    e, psi_out = eng.run()

    outpsis.append(psi_out)
    energies.append(e)

/Users/kierancooney/.pyenv/versions/num_spt_venv_p11/lib/python3.11/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for config TwoSiteDMRGEngine:
['amplitude', 'decay', 'disable_after', 'update_env']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))
/Users/kierancooney/.pyenv/versions/num_spt_venv_p11/lib/python3.11/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for config TwoSiteDMRGEngine:
['amplitude', 'decay', 'disable_after', 'update_env']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))
/Users/kierancooney/.pyenv/versions/num_spt_venv_p11/lib/python3.11/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for config TwoSiteDMRGEngine:
['amplitude', 'decay', 'disable_after', 'update_env']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))


In [19]:
psis[0]

In [20]:
psis[0].get_B(0)

<npc.Array shape=(1, 4, 1) labels=['vL', 'p', 'vR']>

In [21]:
energies

[0.0, 0.0, 0.0, 0.0]

## Non-trivial proj rep

In [14]:
pauli_proj_rep_phases = np.array([
    [1, 1, 1, 1],
    [1, 1, -1, -1],
    [1, -1, 1, -1],
    [1, -1, -1, 1]
])

In [15]:
def get_proj_rep_term_phase(group_quad):
    g_left, g_in, g_out, g_right = group_quad

    g1 = mod_4_to_bit_addition(g_left, g_in)
    g2 = mod_4_to_bit_addition(g_in, g_right)
    g3 = mod_4_to_bit_addition(g_left, g_out)
    g4 = mod_4_to_bit_addition(g_out, g_right)

    numerator_phase = pauli_proj_rep_phases[g1, g2]
    denominator_phase = pauli_proj_rep_phases[g3, g4]

    return numerator_phase/denominator_phase

In [16]:
class ClusterFermion(CouplingMPOModel):
        default_lattice = "Chain"
        force_default_lattice = True

        # Would it be easier to subclass ClockSite?
        def init_sites(self, model_params):
            spin = ClockSite(4, conserve=None)
            for i in range(4):
                for j in range(4):
                    X = np.zeros((4,4))
                    X[j, i] = 1
                    op_name = f"map_{i}_{j}"
                    spin.add_op(op_name, X)
            ferm = FermionSite(conserve=None)
            sites = [spin, ferm]
            return [spin, ferm], ['s', 'f']

        def init_terms(self, model_params):
            # Read off model parameters
            n1_01, n1_10 = model_params.get('n1', (0,0))
            n1_func = get_n1_func(n1_01, n1_10)

            non_trivial_proj_rep = model_params.get('non_trivial_proj_rep', False)

            group_quads = product(range(4), repeat=4)

            for group_quad in group_quads:
                op_list = get_op_list(group_quad, n1_func)
                if non_trivial_proj_rep:
                    phase = get_proj_rep_term_phase(group_quad)
                else:
                    phase = 1
                self.add_multi_coupling((-1/4)*phase, op_list)

In [17]:
M = ClusterIsingFermion({'L': 20})

In [18]:
n1_pairs = [
    (0, 0),
    (0, 1),
    (1, 0),
    (1, 1)
]

In [190]:
models = [
    [ClusterFermion({'L': 20, 'n1': p, 'non_trivial_proj_rep': b}) for p in n1_pairs]
    for b in [True, False]
]

In [191]:
n1_pair_to_fermion_operators_dict

{(0, 0): 'Id', (0, 1): 'C', (1, 0): 'Cd', (1, 1): 'N'}

In [192]:
models

[[<__main__.ClusterFermion at 0x177bdb290>,

In [193]:
psis = [
    [MPS.from_lat_product_state(m.lat, [['0', 'empty'],]*20) for m in l]
    for l in models
]

In [194]:
dmrg_params = {
    "trunc_params": {"chi_max": 8, "chi_min": 1, "svd_min": 1.e-10},
    "min_sweeps":100,
    "max_sweeps":200,
    "mixer": True,
    "combine":False,
    'decay':2,
    'amplitude':10e-1,
    'disable_after':60,
    'update_env':0
}

In [199]:
for l in models:
    for m in l:
        mpo = m.calc_H_MPO()
        print(mpo.is_hermitian())

True
False
False
False
True
False
False
False


In [201]:
out = list()

n1_01, n1_10 = 1, 0
n1_func = get_n1_func(n1_01, n1_10)

non_trivial_proj_rep = False

g_l = 0
g_r = 0

group_pairs = product(range(4), repeat=2)

for group_pair in group_pairs:
    op_list = get_op_list((g_l, *group_pair, g_r), n1_func)
    out.append(op_list)

In [204]:
out

[[('map_0_0', 0, 0),
  ('Id', 0, 1),
  ('map_0_0', 1, 0),
  ('Id', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_0_0', 0, 0),
  ('C', 0, 1),
  ('map_0_1', 1, 0),
  ('C', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_0_0', 0, 0),
  ('Id', 0, 1),
  ('map_0_2', 1, 0),
  ('Id', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_0_0', 0, 0),
  ('C', 0, 1),
  ('map_0_3', 1, 0),
  ('C', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_0_0', 0, 0),
  ('Cd', 0, 1),
  ('map_1_0', 1, 0),
  ('Cd', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_0_0', 0, 0),
  ('N', 0, 1),
  ('map_1_1', 1, 0),
  ('N', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_0_0', 0, 0),
  ('Cd', 0, 1),
  ('map_1_2', 1, 0),
  ('Cd', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_0_0', 0, 0),
  ('N', 0, 1),
  ('map_1_3', 1, 0),
  ('N', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_0_0', 0, 0),
  ('Id', 0, 1),
  ('map_2_0', 1, 0),
  ('Id', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_0_0', 0, 0),
  ('C', 0, 1),
  ('map_2_1', 1, 0),
  ('C', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_0_0', 0, 0),
  ('Id', 0, 1),
  ('map_2_2',

In [208]:
fermion_operators = [l[1][0] for l in out]

In [210]:
[
    fermion_operators[:4],
    fermion_operators[4:8],
    fermion_operators[8:12],
    fermion_operators[12:16]
]

[['Id', 'C', 'Id', 'C'],
 ['Cd', 'N', 'Cd', 'N'],
 ['Id', 'C', 'Id', 'C'],
 ['Cd', 'N', 'Cd', 'N']]

In [211]:
out = list()

n1_01, n1_10 = 1, 0
n1_func = get_n1_func(n1_01, n1_10)

non_trivial_proj_rep = False

g_l = 1
g_r = 0

group_pairs = product(range(4), repeat=2)

for group_pair in group_pairs:
    op_list = get_op_list((g_l, *group_pair, g_r), n1_func)
    out.append(op_list)

In [224]:
out

[[('map_1_1', 0, 0),
  ('N', 0, 1),
  ('map_0_0', 1, 0),
  ('Id', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_1_1', 0, 0),
  ('Cd', 0, 1),
  ('map_0_1', 1, 0),
  ('C', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_1_1', 0, 0),
  ('N', 0, 1),
  ('map_0_2', 1, 0),
  ('Id', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_1_1', 0, 0),
  ('Cd', 0, 1),
  ('map_0_3', 1, 0),
  ('C', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_1_1', 0, 0),
  ('C', 0, 1),
  ('map_1_0', 1, 0),
  ('Cd', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_1_1', 0, 0),
  ('Id', 0, 1),
  ('map_1_1', 1, 0),
  ('N', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_1_1', 0, 0),
  ('C', 0, 1),
  ('map_1_2', 1, 0),
  ('Cd', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_1_1', 0, 0),
  ('Id', 0, 1),
  ('map_1_3', 1, 0),
  ('N', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_1_1', 0, 0),
  ('N', 0, 1),
  ('map_2_0', 1, 0),
  ('Id', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_1_1', 0, 0),
  ('Cd', 0, 1),
  ('map_2_1', 1, 0),
  ('C', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_1_1', 0, 0),
  ('N', 0, 1),
  ('map_2_2', 

In [226]:
[
    (l[1][0], l[2][0], l[3][0])
    for l in out
]

[('N', 'map_0_0', 'Id'),
 ('Cd', 'map_0_1', 'C'),
 ('N', 'map_0_2', 'Id'),
 ('Cd', 'map_0_3', 'C'),
 ('C', 'map_1_0', 'Cd'),
 ('Id', 'map_1_1', 'N'),
 ('C', 'map_1_2', 'Cd'),
 ('Id', 'map_1_3', 'N'),
 ('N', 'map_2_0', 'Id'),
 ('Cd', 'map_2_1', 'C'),
 ('N', 'map_2_2', 'Id'),
 ('Cd', 'map_2_3', 'C'),
 ('C', 'map_3_0', 'Cd'),
 ('Id', 'map_3_1', 'N'),
 ('C', 'map_3_2', 'Cd'),
 ('Id', 'map_3_3', 'N')]

In [228]:
left_fermion_operators = [l[1][0] for l in out]
right_fermion_operators = [l[3][0] for l in out]

In [229]:
[left_fermion_operators[4*i:4*i+4] for i in range(4)]

[['N', 'Cd', 'N', 'Cd'],
 ['C', 'Id', 'C', 'Id'],
 ['N', 'Cd', 'N', 'Cd'],
 ['C', 'Id', 'C', 'Id']]

In [230]:
[right_fermion_operators[4*i:4*i+4] for i in range(4)]

[['Id', 'C', 'Id', 'C'],
 ['Cd', 'N', 'Cd', 'N'],
 ['Id', 'C', 'Id', 'C'],
 ['Cd', 'N', 'Cd', 'N']]

### Check hermiticity

In [216]:
class ClusterFermionHermiticityTest(CouplingMPOModel):
        default_lattice = "Chain"
        force_default_lattice = True

        # Would it be easier to subclass ClockSite?
        def init_sites(self, model_params):
            spin = ClockSite(4, conserve=None)
            for i in range(4):
                for j in range(4):
                    X = np.zeros((4,4))
                    X[j, i] = 1
                    op_name = f"map_{i}_{j}"
                    spin.add_op(op_name, X)
            ferm = FermionSite(conserve=None)
            sites = [spin, ferm]
            return [spin, ferm], ['s', 'f']

        def init_terms(self, model_params):
            # Read off model parameters
            n1_01, n1_10 = model_params.get('n1', (0,0))
            n1_func = get_n1_func(n1_01, n1_10)
            
            g_l = model_params.get('g_l', 0)
            g_r = model_params.get('g_r', 0)

            non_trivial_proj_rep = model_params.get('non_trivial_proj_rep', False)

            group_pairs = product(range(4), repeat=2)

            for group_pair in group_pairs:
                op_list = get_op_list((g_l, *group_pair, g_r), n1_func)
                if non_trivial_proj_rep:
                    phase = get_proj_rep_term_phase(group_quad)
                else:
                    phase = 1
                self.add_multi_coupling((-1/4)*phase, op_list)

In [217]:
models = [
    [
        [ClusterFermionHermiticityTest({'L': 20, 'n1': p, 'g_l': g_l, 'g_r': g_r}) for p in n1_pairs]
        for g_l in range(4)
    ]
    for g_r in range(4)
]

In [220]:
mpos = [[[m.calc_H_MPO() for m in l1] for l1 in l2] for l2 in models]

In [221]:
[[[m.is_hermitian() for m in l1] for l1 in l2] for l2 in mpos]

[[[True, False, False, False],
  [True, False, False, False],
  [True, False, False, False],
  [True, False, False, False]],
 [[True, False, False, False],
  [True, False, False, False],
  [True, False, False, False],
  [True, False, False, False]],
 [[True, False, False, False],
  [True, False, False, False],
  [True, False, False, False],
  [True, False, False, False]],
 [[True, False, False, False],
  [True, False, False, False],
  [True, False, False, False],
  [True, False, False, False]]]

In [223]:
FermionSite().need_JW_string

{'C', 'Cd', 'JW'}

In [208]:
fermion_operators = [l[1][0] for l in out]

In [210]:
[
    fermion_operators[:4],
    fermion_operators[4:8],
    fermion_operators[8:12],
    fermion_operators[12:16]
]

[['Id', 'C', 'Id', 'C'],
 ['Cd', 'N', 'Cd', 'N'],
 ['Id', 'C', 'Id', 'C'],
 ['Cd', 'N', 'Cd', 'N']]

### Break down into further subcases

In [279]:
class ClusterFermionHermiticityTest(CouplingMPOModel):
    default_lattice = "Chain"
    force_default_lattice = True

    # Would it be easier to subclass ClockSite?
    def init_sites(self, model_params):
        spin = ClockSite(4, conserve=None)
        for i in range(4):
            for j in range(4):
                X = np.zeros((4,4))
                X[j, i] = 1
                op_name = f"map_{i}_{j}"
                spin.add_op(op_name, X)
        ferm = FermionSite(conserve=None)
        sites = [spin, ferm]
        return [spin, ferm], ['s', 'f']

    def init_terms(self, model_params):
        # Read off model parameters
        n1_01, n1_10 = model_params.get('n1', (0,0))
        n1_func = get_n1_func(n1_01, n1_10)

        g_l = model_params.get('g_l', 0)
        g_r = model_params.get('g_r', 0)
        g_in = model_params.get('g_in', 0)
        g_out = model_params.get('g_out', 0)

        op_list_1 = get_op_list((g_l, g_in, g_out, g_r), n1_func)
        self.add_multi_coupling((-1/4), op_list_1)
        
        op_list_2 = get_op_list((g_l, g_out, g_in, g_r), n1_func)
        self.add_multi_coupling((-1/4), op_list_2)

In [280]:
models = list()

for p in n1_pairs:
    models.append(list())
    for g_l in range(4):
        models[-1].append(list())
        for g_r in range(4):
            models[-1][-1].append(list())
            for g_in in range(4):
                models[-1][-1][-1].append(list())   
                for g_out in range(4):
                    m = ClusterFermionHermiticityTest(
                        {'L': 20, 'n1': p, 'g_l': g_l, 'g_r': g_r, 'g_in': g_in, 'g_out': g_out}
                    )
                    
                    models[-1][-1][-1][-1].append(m)

In [281]:
models[0][0][0][0][0]

In [282]:
mpos = [[[[[m.calc_H_MPO() for m in l1] for l1 in l2] for l2 in l3] for l3 in l4] for l4 in models]

In [283]:
is_hermitian = np.array([[[[[m.is_hermitian() for m in l1] for l1 in l2] for l2 in l3] for l3 in l4] for l4 in mpos])

In [284]:
is_hermitian.shape

(4, 4, 4, 4, 4)

In [302]:
is_hermitian[1]

array([[[[ True,  True, False, False],
         [ True,  True, False, False],
         [False, False,  True,  True],
         [False, False,  True,  True]],

        [[ True,  True, False, False],
         [ True,  True, False, False],
         [False, False,  True,  True],
         [False, False,  True,  True]],

        [[ True,  True, False, False],
         [ True,  True, False, False],
         [False, False,  True,  True],
         [False, False,  True,  True]],

        [[ True,  True, False, False],
         [ True,  True, False, False],
         [False, False,  True,  True],
         [False, False,  True,  True]]],


       [[[ True,  True, False, False],
         [ True,  True, False, False],
         [False, False,  True,  True],
         [False, False,  True,  True]],

        [[ True,  True, False, False],
         [ True,  True, False, False],
         [False, False,  True,  True],
         [False, False,  True,  True]],

        [[ True,  True, False, False],
         [ 

So breakdown in hermiticity is a function of $n_1$, $g_{in}$ and $g_{out}$, independent of $g_l$ and $g_r$.

So hermitian only if $g_{in}$ and $g_{out}$ are equal... i.e. it's the creation and annihilation terms causing problems.

In [310]:
n1_func = get_n1_func(0, 1)

g_l = 0
g_r = 0
g_in = 0
g_out = 2

In [321]:
get_op_list((g_l, 3, 3, g_r), n1_func)

[('map_0_0', 0, 0),
 ('N', 0, 1),
 ('map_3_3', 1, 0),
 ('N', 1, 1),
 ('map_0_0', 2, 0)]

In [322]:
op_list_1 = get_op_list((g_l, g_in, g_out, g_r), n1_func)

In [323]:
op_list_1

[('map_0_0', 0, 0),
 ('C', 0, 1),
 ('map_0_2', 1, 0),
 ('C', 1, 1),
 ('map_0_0', 2, 0)]

In [324]:
op_list_2 = get_op_list((g_l, g_out, g_in, g_r), n1_func)

In [325]:
op_list_2

[('map_0_0', 0, 0),
 ('Cd', 0, 1),
 ('map_2_0', 1, 0),
 ('Cd', 1, 1),
 ('map_0_0', 2, 0)]

Extract terms?

In [326]:
terms = models[1][0][0][2][0].all_coupling_terms()

In [327]:
terms

In [328]:
term_list = terms.to_TermList()

In [330]:
tl = term_list.terms

In [331]:
len(tl)

36

In [332]:
tl[0], tl[18]

([('map_0_0', 0), ('Cd JW', 1), ('map_2_0 JW', 2), ('Cd', 3), ('map_0_0', 4)],
 [('map_0_0', 0), ('C JW', 1), ('map_0_2 JW', 2), ('C', 3), ('map_0_0', 4)])

### Repeat but add hermitian conjugate terms

In [333]:
class ClusterFermionHermiticityTest(CouplingMPOModel):
    default_lattice = "Chain"
    force_default_lattice = True

    # Would it be easier to subclass ClockSite?
    def init_sites(self, model_params):
        spin = ClockSite(4, conserve=None)
        for i in range(4):
            for j in range(4):
                X = np.zeros((4,4))
                X[j, i] = 1
                op_name = f"map_{i}_{j}"
                spin.add_op(op_name, X)
        ferm = FermionSite(conserve=None)
        sites = [spin, ferm]
        return [spin, ferm], ['s', 'f']

    def init_terms(self, model_params):
        # Read off model parameters
        n1_01, n1_10 = model_params.get('n1', (0,0))
        n1_func = get_n1_func(n1_01, n1_10)

        g_l = model_params.get('g_l', 0)
        g_r = model_params.get('g_r', 0)
        g_in = model_params.get('g_in', 0)
        g_out = model_params.get('g_out', 0)

        op_list_1 = get_op_list((g_l, g_in, g_out, g_r), n1_func)
        self.add_multi_coupling((-1/4), op_list_1, plus_hc=True)
        
        op_list_2 = get_op_list((g_l, g_out, g_in, g_r), n1_func)
        self.add_multi_coupling((-1/4), op_list_2, plus_hc=True)

In [339]:
models = list()
none_count = 0

for p in n1_pairs:
    models.append(list())
    for g_l in range(4):
        models[-1].append(list())
        for g_r in range(4):
            models[-1][-1].append(list())
            for g_in in range(4):
                models[-1][-1][-1].append(list())   
                for g_out in range(4):
                    try:
                        m = ClusterFermionHermiticityTest(
                            {'L': 20, 'n1': p, 'g_l': g_l, 'g_r': g_r, 'g_in': g_in, 'g_out': g_out}
                        )
                    except ValueError:
                        m = None
                        none_count += 1
                    
                    models[-1][-1][-1][-1].append(m)

In [340]:
none_count

384

$384 = 128*3 \Rightarrow 1024/384 = 8/3$

Which ones are breaking...

In [341]:
models[0][0][0][0][0]

In [342]:
mpos = [[[[[None if m is None else m.calc_H_MPO() for m in l1] for l1 in l2] for l2 in l3] for l3 in l4] for l4 in models]

In [343]:
none_array = np.array([[[[[m is None for m in l1] for l1 in l2] for l2 in l3] for l3 in l4] for l4 in models])

Looks like the cases I'm interested in are exactly those missing...!

In [349]:
none_array[1]

array([[[[False, False,  True,  True],
         [False, False,  True,  True],
         [ True,  True, False, False],
         [ True,  True, False, False]],

        [[False, False,  True,  True],
         [False, False,  True,  True],
         [ True,  True, False, False],
         [ True,  True, False, False]],

        [[False, False,  True,  True],
         [False, False,  True,  True],
         [ True,  True, False, False],
         [ True,  True, False, False]],

        [[False, False,  True,  True],
         [False, False,  True,  True],
         [ True,  True, False, False],
         [ True,  True, False, False]]],


       [[[False, False,  True,  True],
         [False, False,  True,  True],
         [ True,  True, False, False],
         [ True,  True, False, False]],

        [[False, False,  True,  True],
         [False, False,  True,  True],
         [ True,  True, False, False],
         [ True,  True, False, False]],

        [[False, False,  True,  True],
         [F

In [223]:
FermionSite().need_JW_string

{'C', 'Cd', 'JW'}

In [208]:
fermion_operators = [l[1][0] for l in out]

In [210]:
[
    fermion_operators[:4],
    fermion_operators[4:8],
    fermion_operators[8:12],
    fermion_operators[12:16]
]

[['Id', 'C', 'Id', 'C'],
 ['Cd', 'N', 'Cd', 'N'],
 ['Id', 'C', 'Id', 'C'],
 ['Cd', 'N', 'Cd', 'N']]

In [ ]:
models[0][1].c

In [195]:
for l in psis:
    for psi in l:
        psi.canonical_form()

In [1]:
outpsis = list()
energies = list()

for l1, l2 in zip(psis, models):
    outpsis.append(list())
    energies.append(list())
    
    for psi, m in zip(l1, l2):
        # Commenting out to supress error message clutter in notebook.
        """
        eng = dmrg.TwoSiteDMRGEngine(psi, m, dmrg_params)
        e, psi_out = eng.run()

        outpsis[-1].append(psi_out)
        energies[-1].append(e)
        """

NameError: name 'psis' is not defined

In [197]:
energies

[[-18.000000000000036,
  -8.196875831824364,
  -8.514815527402932,
  -8.97384686708272],
 [-18.00000000000004,
  -8.628884646999314,
  -8.279348259530973,
  -8.999999999999979]]

## ZXZ case

In [105]:
def get_xzx_op_list(g_quad):
    out = list()
    g_left, g_in, g_out, g_right = g_quad

    out.append((f'map_{g_left}_{g_left}', 0, 0))
    
    out.append((f'map_{g_in}_{g_out}', 1, 0))
    
    out.append((f'map_{g_right}_{g_right}', 2, 0))

    return out

In [136]:
class Cluster(CouplingMPOModel):
        default_lattice = "Chain"
        force_default_lattice = True

        # Would it be easier to subclass ClockSite?
        def init_sites(self, model_params):
            spin = ClockSite(4, conserve=None)
            for i in range(4):
                for j in range(4):
                    X = np.zeros((4,4))
                    X[j, i] = 1
                    op_name = f"map_{i}_{j}"
                    spin.add_op(op_name, X)
            sites = [spin,]
            return [spin,], ['s',]

        def init_terms(self, model_params):
            # Read off model parameters
            
            non_trivial_proj_rep = model_params.get('non_trivial_proj_rep', False)

            group_quads = product(range(4), repeat=4)

            for group_quad in group_quads:
                op_list = get_xzx_op_list(group_quad)
                if non_trivial_proj_rep:
                    phase = get_proj_rep_term_phase(group_quad)
                else:
                    phase = 1
                self.add_multi_coupling((-1/4)*phase, op_list)

In [137]:
models = [
    Cluster({'L': 20, 'non_trivial_proj_rep': b})
    for b in [True, False]
]

In [138]:
models

[<__main__.Cluster at 0x169fe90d0>, <__main__.Cluster at 0x168990690>]

In [139]:
psis = [
    MPS.from_lat_product_state(m.lat, [['0',],]*20) for m in models
]

In [140]:
dmrg_params = {
    "trunc_params": {"chi_max": 8, "chi_min": 1, "svd_min": 1.e-10},
    "min_sweeps":100,
    "max_sweeps":200,
    "mixer": True,
    "combine":False,
    'decay':2,
    'amplitude':10e-1,
    'disable_after':60,
    'update_env':0
}

In [141]:
for psi in psis:
    psi.canonical_form()

In [142]:
outpsis = list()
energies = list()

for psi, m in zip(psis, models):
    eng = dmrg.TwoSiteDMRGEngine(psi, m, dmrg_params)
    e, psi_out = eng.run()

    outpsis.append(psi_out)
    energies.append(e)

/Users/kierancooney/.pyenv/versions/num_spt_venv_p11/lib/python3.11/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for config TwoSiteDMRGEngine:
['amplitude', 'decay', 'disable_after', 'update_env']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))


In [143]:
energies

[-17.999999999999996, -18.000000000000046]

In [144]:
models

[<__main__.Cluster at 0x169fe90d0>, <__main__.Cluster at 0x168990690>]

In [152]:
m = models[0]

In [153]:
m.test_sanity()

In [154]:
mpo = m.calc_H_MPO()

In [155]:
mpo.is_hermitian()

True

In [156]:
m = models[1]

In [157]:
m.test_sanity()

In [158]:
mpo = m.calc_H_MPO()

In [159]:
mpo.is_hermitian()

True

In [160]:
psi = outpsis[1]

In [163]:
psi.get_B(0).to_ndarray()

array([[[1.],
        [0.],
        [0.],
        [0.]]])

In [164]:
psi.get_B(1).to_ndarray()

array([[[-0.5],
        [-0.5],
        [-0.5],
        [-0.5]]])

In [165]:
psi.get_B(10).to_ndarray()

array([[[-0.5],
        [-0.5],
        [-0.5],
        [-0.5]]])

# Update hamiltonian

In [18]:
def get_op_list_1(g_quad, n1_func):
    out = list()
    g_left, g_in, g_out, g_right = g_quad

    out.append((f'map_{g_left}_{g_left}', 0, 0))

    if n1_func(g_in, g_right):
        out.append(('Cd', 1, 1))
    if n1_func(g_left, g_in):
        out.append(('Cd', 0, 1))
    
    out.append((f'map_{g_in}_{g_out}', 1, 0))
    
    if n1_func(g_left, g_out):
        out.append(('C', 0, 1))
    if n1_func(g_out, g_right):
        out.append(('C', 1, 1))

    out.append((f'map_{g_right}_{g_right}', 2, 0))

    return out

In [19]:
class ClusterFermion1(CouplingMPOModel):
        default_lattice = "Chain"
        force_default_lattice = True

        # Would it be easier to subclass ClockSite?
        def init_sites(self, model_params):
            spin = ClockSite(4, conserve=None)
            for i in range(4):
                for j in range(4):
                    X = np.zeros((4,4))
                    X[j, i] = 1
                    op_name = f"map_{i}_{j}"
                    spin.add_op(op_name, X)
            ferm = FermionSite(conserve=None)
            sites = [spin, ferm]
            return [spin, ferm], ['s', 'f']

        def init_terms(self, model_params):
            # Read off model parameters
            n1_01, n1_10 = model_params.get('n1', (0,0))
            n1_func = get_n1_func(n1_01, n1_10)

            non_trivial_proj_rep = model_params.get('non_trivial_proj_rep', False)

            group_quads = product(range(4), repeat=4)

            for group_quad in group_quads:
                op_list = get_op_list_1(group_quad, n1_func)
                
                if non_trivial_proj_rep:
                    phase = get_proj_rep_term_phase(group_quad)
                else:
                    phase = 1

                g_left, *_, g_right = group_quad
                if n1_func(g_left, g_right) == 1:
                    factor = 2
                else:
                    factor = 1

                self.add_multi_coupling((-1/4)*phase*factor, op_list)

In [23]:
models = [
    [ClusterFermion1({'L': 20, 'n1': p, 'non_trivial_proj_rep': b}) for p in n1_pairs]
    for b in [True, False]
]

In [25]:
models

[[<__main__.ClusterFermion1 at 0x7855c3d72210>,

In [26]:
psis = [
    [MPS.from_lat_product_state(m.lat, [['0', 'empty'],]*20) for m in l]
    for l in models
]

In [27]:
dmrg_params = {
    "trunc_params": {"chi_max": 8, "chi_min": 1, "svd_min": 1.e-10},
    "min_sweeps":100,
    "max_sweeps":200,
    "mixer": True,
    "combine":False,
    'decay':2,
    'amplitude':10e-1,
    'disable_after':60,
    'update_env':0
}

In [28]:
for l in models:
    for m in l:
        mpo = m.calc_H_MPO()
        print(mpo.is_hermitian())

True
True
True
True
True
True
True
True


In [29]:
for l in psis:
    for psi in l:
        psi.canonical_form()

In [30]:
outpsis = list()
energies = list()

for l1, l2 in zip(psis, models):
    outpsis.append(list())
    energies.append(list())

        
    for psi, m in zip(l1, l2):
        eng = dmrg.TwoSiteDMRGEngine(psi, m, dmrg_params)
        e, psi_out = eng.run()
    
        outpsis[-1].append(psi_out)
        energies[-1].append(e)

/home/kieranc/Desktop/numerical_spt_classification/code/num_spt_venv_p11/lib/python3.11/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for config TwoSiteDMRGEngine:
['amplitude', 'decay', 'disable_after', 'update_env']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))
/home/kieranc/Desktop/numerical_spt_classification/code/num_spt_venv_p11/lib/python3.11/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for config TwoSiteDMRGEngine:
['amplitude', 'decay', 'disable_after', 'update_env']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))
/home/kieranc/Desktop/numerical_spt_classification/code/num_spt_venv_p11/lib/python3.11/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for config TwoSiteDMRGEngine:
['amplitude', 'decay', 'disable_after', 'update_env']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))
/home/kieranc/Desktop/numerical_spt_classification/code/num_spt_venv_p11/lib/pytho

In [31]:
energies

[[-17.999999999999982,
  -17.999999999999975,
  -17.999999999999964,
  -17.99999999999994],
 [-18.000000000000032,
  -18.000000000000004,
  -17.999999999999975,
  -17.99999999999997]]

Try without "factor"

In [32]:
class ClusterFermion1(CouplingMPOModel):
        default_lattice = "Chain"
        force_default_lattice = True

        # Would it be easier to subclass ClockSite?
        def init_sites(self, model_params):
            spin = ClockSite(4, conserve=None)
            for i in range(4):
                for j in range(4):
                    X = np.zeros((4,4))
                    X[j, i] = 1
                    op_name = f"map_{i}_{j}"
                    spin.add_op(op_name, X)
            ferm = FermionSite(conserve=None)
            sites = [spin, ferm]
            return [spin, ferm], ['s', 'f']

        def init_terms(self, model_params):
            # Read off model parameters
            n1_01, n1_10 = model_params.get('n1', (0,0))
            n1_func = get_n1_func(n1_01, n1_10)

            non_trivial_proj_rep = model_params.get('non_trivial_proj_rep', False)

            group_quads = product(range(4), repeat=4)

            for group_quad in group_quads:
                op_list = get_op_list_1(group_quad, n1_func)
                
                if non_trivial_proj_rep:
                    phase = get_proj_rep_term_phase(group_quad)
                else:
                    phase = 1

                g_left, *_, g_right = group_quad
                factor=1

                self.add_multi_coupling((-1/4)*phase*factor, op_list)

In [33]:
models = [
    [ClusterFermion1({'L': 20, 'n1': p, 'non_trivial_proj_rep': b}) for p in n1_pairs]
    for b in [True, False]
]

In [34]:
models

[[<__main__.ClusterFermion1 at 0x7855bd747390>,

In [35]:
psis = [
    [MPS.from_lat_product_state(m.lat, [['0', 'empty'],]*20) for m in l]
    for l in models
]

In [36]:
dmrg_params = {
    "trunc_params": {"chi_max": 8, "chi_min": 1, "svd_min": 1.e-10},
    "min_sweeps":100,
    "max_sweeps":200,
    "mixer": True,
    "combine":False,
    'decay':2,
    'amplitude':10e-1,
    'disable_after':60,
    'update_env':0
}

In [37]:
for l in models:
    for m in l:
        mpo = m.calc_H_MPO()
        print(mpo.is_hermitian())

True
True
True
True
True
True
True
True


In [38]:
for l in psis:
    for psi in l:
        psi.canonical_form()

In [39]:
outpsis = list()
energies = list()

for l1, l2 in zip(psis, models):
    outpsis.append(list())
    energies.append(list())

        
    for psi, m in zip(l1, l2):
        eng = dmrg.TwoSiteDMRGEngine(psi, m, dmrg_params)
        e, psi_out = eng.run()
    
        outpsis[-1].append(psi_out)
        energies[-1].append(e)

/home/kieranc/Desktop/numerical_spt_classification/code/num_spt_venv_p11/lib/python3.11/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for config TwoSiteDMRGEngine:
['amplitude', 'decay', 'disable_after', 'update_env']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))
/home/kieranc/Desktop/numerical_spt_classification/code/num_spt_venv_p11/lib/python3.11/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for config TwoSiteDMRGEngine:
['amplitude', 'decay', 'disable_after', 'update_env']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))
/home/kieranc/Desktop/numerical_spt_classification/code/num_spt_venv_p11/lib/python3.11/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for config TwoSiteDMRGEngine:
['amplitude', 'decay', 'disable_after', 'update_env']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))
/home/kieranc/Desktop/numerical_spt_classification/code/num_spt_venv_p11/lib/pytho

In [40]:
energies

[[-17.999999999999982,
  -12.342466171618899,
  -12.34246617161892,
  -12.3424661716189],
 [-18.000000000000032,
  -12.342052318754007,
  -12.342146563149196,
  -12.341908845734233]]

## Check correlation lengths
Should be 0.

In [19]:
def get_op_list_1(g_quad, n1_func):
    out = list()
    g_left, g_in, g_out, g_right = g_quad

    out.append((f'map_{g_left}_{g_left}', 0, 0))

    if n1_func(g_in, g_right):
        out.append(('Cd', 1, 1))
    if n1_func(g_left, g_in):
        out.append(('Cd', 0, 1))
    
    out.append((f'map_{g_in}_{g_out}', 1, 0))
    
    if n1_func(g_left, g_out):
        out.append(('C', 0, 1))
    if n1_func(g_out, g_right):
        out.append(('C', 1, 1))

    out.append((f'map_{g_right}_{g_right}', 2, 0))

    return out

In [20]:
class ClusterFermion1(CouplingMPOModel):
        default_lattice = "Chain"
        force_default_lattice = True

        # Would it be easier to subclass ClockSite?
        def init_sites(self, model_params):
            spin = ClockSite(4, conserve=None)
            for i in range(4):
                for j in range(4):
                    X = np.zeros((4,4))
                    X[j, i] = 1
                    op_name = f"map_{i}_{j}"
                    spin.add_op(op_name, X)
            ferm = FermionSite(conserve=None)
            sites = [spin, ferm]
            return [spin, ferm], ['s', 'f']

        def init_terms(self, model_params):
            # Read off model parameters
            n1_01, n1_10 = model_params.get('n1', (0,0))
            n1_func = get_n1_func(n1_01, n1_10)

            non_trivial_proj_rep = model_params.get('non_trivial_proj_rep', False)

            group_quads = product(range(4), repeat=4)

            for group_quad in group_quads:
                op_list = get_op_list_1(group_quad, n1_func)
                
                if non_trivial_proj_rep:
                    phase = get_proj_rep_term_phase(group_quad)
                else:
                    phase = 1

                g_left, *_, g_right = group_quad
                if n1_func(g_left, g_right) == 1:
                    factor = 2
                else:
                    factor = 1

                self.add_multi_coupling((-1/4)*phase*factor, op_list)

In [21]:
models = [
    [ClusterFermion1({'L': 50, 'n1': p, 'non_trivial_proj_rep': b}) for p in n1_pairs]
    for b in [True, False]
]

In [22]:
models

[[<__main__.ClusterFermion1 at 0x152e44350>,

In [23]:
psis = [
    [MPS.from_lat_product_state(m.lat, [['0', 'empty'],]*50) for m in l]
    for l in models
]

In [24]:
dmrg_params = {
    "trunc_params": {"chi_max": 8, "chi_min": 1, "svd_min": 1.e-10},
    "min_sweeps":100,
    "max_sweeps":200,
    "mixer": True,
    "combine":False,
    'decay':2,
    'amplitude':10e-1,
    'disable_after':60,
    'update_env':0
}

In [25]:
for l in psis:
    for psi in l:
        psi.canonical_form()

In [26]:
outpsis = list()
energies = list()

for l1, l2 in zip(psis, models):
    outpsis.append(list())
    energies.append(list())

        
    for psi, m in zip(l1, l2):
        try:
            eng = dmrg.TwoSiteDMRGEngine(psi, m, dmrg_params)
            e, psi_out = eng.run()
        except:
            e, psi_out = None, None

        outpsis[-1].append(psi_out)
        energies[-1].append(e)

/Users/kierancooney/.pyenv/versions/num_spt_venv_p11/lib/python3.11/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for config TwoSiteDMRGEngine:
['amplitude', 'decay', 'disable_after', 'update_env']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))
/Users/kierancooney/.pyenv/versions/num_spt_venv_p11/lib/python3.11/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for config TwoSiteDMRGEngine:
['amplitude', 'decay', 'disable_after', 'update_env']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))
/Users/kierancooney/.pyenv/versions/num_spt_venv_p11/lib/python3.11/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for config TwoSiteDMRGEngine:
['amplitude', 'decay', 'disable_after', 'update_env']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))
/Users/kierancooney/.pyenv/versions/num_spt_venv_p11/lib/python3.11/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for c

In [54]:
energies

[[-96.0000000000011,
  -61.932424627532924,
  -61.93242462752965,
  -61.9324246248189],
 [-96.00000000000038,
  -61.93197511895478,
  -61.93197511895475,
  -61.93197511895518]]

In [28]:
len(outpsis[0][0].sites)

100

In [29]:
outpsis[0][0].get_SR(10)

array([0.5, 0.5, 0.5, 0.5])

In [30]:
outpsis[0][0].get_SR(9)

array([0.5, 0.5, 0.5, 0.5])

In [31]:
outpsis[0][0].get_SR(11)

array([0.5, 0.5, 0.5, 0.5])

In [32]:
boson_expectations = np.zeros((2, 2, 4, 50))

indices = product(
    enumerate(['X', 'Z']),
    range(2),
    range(4)
)

for (i, op), j, k in indices:
    try:
        psi = outpsis[j][k]
        
        data = (
            psi
            .expectation_value(
                op,
                list(range(0, 100, 2))
            )
        )

        boson_expectations[i,j,k] = data

    except:
        boson_expectations[i,j,k] = np.nan

/var/folders/r3/xn8xq5c17932m2g1b4r3dss80000gn/T/ipykernel_79511/3510999881.py:21: ComplexWarning: Casting complex values to real discards the imaginary part
  boson_expectations[i,j,k] = data


In [33]:
boson_expectations[0]

array([[[ 0.00000000e+00, -7.79567137e-15, -9.85322934e-16,
         -1.41119755e-15, -5.18422111e-15, -2.42514342e-15,
         -2.40953091e-15,  2.10248485e-15, -8.39606162e-16,
         -8.63892291e-16,  6.50521303e-16,  1.45716772e-16,
          6.31439345e-16, -4.37150316e-16,  6.01949046e-16,
         -2.15626128e-15,  5.10008702e-16,  2.01227923e-15,
         -6.89986263e-16,  1.56125113e-15, -1.28543010e-15,
         -2.28983499e-15,  8.74300632e-16, -4.08006962e-15,
          6.66133815e-16,  3.46944695e-18,  5.69683190e-15,
          5.16947596e-16, -3.98986399e-15,  2.30718222e-16,
          1.58900670e-15,  1.19695920e-15, -6.93889390e-18,
         -8.84708973e-16,  1.87350135e-15,  5.55111512e-17,
          2.57172755e-15, -2.41473508e-15, -1.18655086e-15,
          1.28716482e-15, -2.77555756e-16, -6.24500451e-17,
         -5.55111512e-16,  7.18175519e-16, -9.88792381e-16,
         -1.08246745e-15, -9.43689571e-16, -5.20417043e-16,
          3.03541030e-14,  0.00000000e+0

In [34]:
boson_expectations[1]

array([[[ 1.00000000e+00, -9.21485110e-15, -2.15938378e-14,
         -2.40224507e-14, -1.50157664e-14, -9.60342916e-15,
          2.00256478e-14,  3.32234240e-14,  1.82631688e-14,
         -1.74027459e-14,  1.10744747e-14, -7.75768338e-15,
         -5.38458167e-14, -6.22418783e-15,  2.35922393e-14,
         -2.37032616e-14,  6.68909372e-15, -7.83401122e-15,
         -4.28129754e-14, -2.85882429e-14, -3.16968674e-14,
         -2.16215934e-14, -1.14561138e-14, -2.34881559e-14,
         -2.47024623e-15,  2.24473218e-14, -2.91433544e-15,
         -6.06459327e-15,  1.51476054e-14,  1.38777878e-16,
          4.65599781e-14, -6.58501031e-15, -1.79023463e-14,
         -2.52853294e-14, -2.05668815e-14, -5.14518983e-15,
         -1.88321581e-14,  2.85882429e-15, -2.75890422e-14,
          7.27196081e-15,  1.70974346e-14,  2.03101425e-14,
         -2.74780199e-15, -4.05231404e-15, -2.43832732e-14,
         -1.29063427e-15,  1.27675648e-15, -1.68268177e-15,
         -2.06779038e-14,  1.00000000e+0

In [35]:
fermion_number_expectations = np.zeros((2, 4, 50))

indices = product(
    range(2),
    range(4)
)

for j, k in indices:
    try:
        psi = outpsis[j][k]
        
        data = (
            psi
            .expectation_value(
                'N',
                list(range(1, 100, 2))
            )
        )

        fermion_number_expectations[j,k] = data

    except:
        fermion_number_expectations[j,k] = np.nan

In [36]:
fermion_number_expectations

array([[[1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 2.27256093e-29,
         3.36704143e-31, 5.30820785e-32, 6.21023021e-32, 1.00000000e+00,
         1.00000000e+00, 2.67174290e-28, 1.00000000e+00, 1.00000000e+00,
         1.00000000e+00, 4.73666330e-31, 1.88013136e-31, 2.67383690e-29,
         1.00000000e+00, 1.00000000e+00, 3.66745346e-32, 8.79119808e-32,
         8.07381653e-32, 7.10372603e-32, 1.00000000e+00, 1.00000000e+00,
         2.09629089e-32, 1.57050773e-31, 1.55143935e-31, 1.28249743e-31,
         1.00000000e+00, 1.00000000e+00, 2.02159253e-31, 1.00000000e+00,
         7.46080879e-32, 1.86407726e-27, 1.00000000e+00, 3.40935908e-32,
         1.00000000e+00, 1.00000000e+00, 2.74415578e-32, 2.31492548e-32,
         1.00000000e+00, 8.05597885e-32, 1.19821484e-31, 7.85590297e-32,
         1.00000000e+00, 3.16869088e-31, 1.00000000e+00, 4.09195291e-32,
         1.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 1.00000000e+00, 0.00000000e+00, 1.00000000e+00,
         

In [37]:
boson_correlations = np.zeros((2, 2, 4, 50, 50))

indices = product(
    enumerate(['X', 'Z']),
    range(2),
    range(4)
)

for (i, op), j, k in indices:
    try:
        psi = outpsis[j][k]
        
        data = (
            psi
            .correlation_function(
                op,
                op,
                list(range(0, 100, 2)),
                list(range(0, 100, 2)),
            )
        )
    
        boson_correlations[i,j,k] = data

    except:
        boson_correlations[i,j,k] = np.nan

/var/folders/r3/xn8xq5c17932m2g1b4r3dss80000gn/T/ipykernel_79511/290460682.py:23: ComplexWarning: Casting complex values to real discards the imaginary part
  boson_correlations[i,j,k] = data


In [38]:
boson_correlations[..., 1, -2]

array([[[ 1.20751113e-28,  2.50000000e-01,  0.00000000e+00,
          2.50000000e-01],
        [ 1.00000000e+00,  2.50000000e-01,  0.00000000e+00,
          2.50000000e-01]],

       [[ 3.68792473e-29,  2.26977690e-16, -1.14863880e-32,
         -2.07590842e-16],
        [-2.01883663e-28,  8.98339075e-17, -6.12236204e-31,
         -4.07189559e-17]]])

In [39]:
fermion_correlations = np.zeros((2, 4, 50, 50))

indices = product(
    range(2),
    range(4)
)

for j, k in indices:
    try:
        psi = outpsis[j][k]
        
        data = (
            psi
            .correlation_function(
                'N',
                'N',
                list(range(1, 101, 2)),
                list(range(1, 101, 2)),
            )
        )
    
        fermion_correlations[j,k] = data

    except:
        fermion_correlations[j,k] = np.nan

In [40]:
fermion_correlations[..., 1, -2]

array([[1.        , 0.        , 1.        , 0.91158361],
       [0.11595859, 0.        , 1.        , 1.        ]])

In [41]:
reduced_boson_correlations = (
    boson_correlations -
    boson_expectations[...,np.newaxis]*boson_expectations[...,np.newaxis,:]
)

In [42]:
reduced_boson_correlations[...,1,-2]

array([[[ 3.57381724e-28, -2.44249065e-15,  0.00000000e+00,
         -1.72084569e-15],
        [ 1.33226763e-15, -1.66533454e-15,  0.00000000e+00,
         -6.38378239e-16]],

       [[-1.53664558e-28, -2.50000000e-01, -1.52357875e-32,
         -2.50000000e-01],
        [-1.72271880e-28, -2.50000000e-01, -6.15985603e-31,
         -2.50000000e-01]]])

In [43]:
reduced_fermion_correlations = (
    fermion_correlations -
    fermion_number_expectations[...,np.newaxis]*fermion_number_expectations[...,np.newaxis,:]
)

In [44]:
reduced_fermion_correlations[...,1,-2]

array([[ 9.76996262e-15,  0.00000000e+00, -4.55191440e-15,
        -6.43929354e-15],
       [-2.77555756e-17,  0.00000000e+00, -3.33066907e-15,
        -2.99760217e-15]])

## Correct possible factor issue.
Think I may have made a mistake... again.

In [45]:
def get_op_list_1(g_quad, n1_func):
    out = list()
    g_left, g_in, g_out, g_right = g_quad

    out.append((f'map_{g_left}_{g_left}', 0, 0))

    if n1_func(g_in, g_right):
        out.append(('Cd', 1, 1))
    if n1_func(g_left, g_in):
        out.append(('Cd', 0, 1))
    
    out.append((f'map_{g_in}_{g_out}', 1, 0))
    
    if n1_func(g_left, g_out):
        out.append(('C', 0, 1))
    if n1_func(g_out, g_right):
        out.append(('C', 1, 1))

    out.append((f'map_{g_right}_{g_right}', 2, 0))

    return out

In [46]:
class ClusterFermion1(CouplingMPOModel):
        default_lattice = "Chain"
        force_default_lattice = True

        # Would it be easier to subclass ClockSite?
        def init_sites(self, model_params):
            spin = ClockSite(4, conserve=None)
            for i in range(4):
                for j in range(4):
                    X = np.zeros((4,4))
                    X[j, i] = 1
                    op_name = f"map_{i}_{j}"
                    spin.add_op(op_name, X)
            ferm = FermionSite(conserve=None)
            sites = [spin, ferm]
            return [spin, ferm], ['s', 'f']

        def init_terms(self, model_params):
            # Read off model parameters
            n1_01, n1_10 = model_params.get('n1', (0,0))
            n1_func = get_n1_func(n1_01, n1_10)

            non_trivial_proj_rep = model_params.get('non_trivial_proj_rep', False)

            group_quads = product(range(4), repeat=4)

            for group_quad in group_quads:
                op_list = get_op_list_1(group_quad, n1_func)
                
                if non_trivial_proj_rep:
                    phase = get_proj_rep_term_phase(group_quad)
                else:
                    phase = 1

                g_left, *_, g_right = group_quad
                if n1_func(g_left, g_right) == 1:
                    factor = 1
                else:
                    factor = 2

                self.add_multi_coupling((-1/4)*phase*factor, op_list)

In [47]:
models = [
    [ClusterFermion1({'L': 50, 'n1': p, 'non_trivial_proj_rep': b}) for p in n1_pairs]
    for b in [True, False]
]

In [48]:
models

[[<__main__.ClusterFermion1 at 0x17732e210>,

In [49]:
psis = [
    [MPS.from_lat_product_state(m.lat, [['0', 'empty'],]*50) for m in l]
    for l in models
]

In [50]:
dmrg_params = {
    "trunc_params": {"chi_max": 8, "chi_min": 1, "svd_min": 1.e-10},
    "min_sweeps":100,
    "max_sweeps":200,
    "mixer": True,
    "combine":False,
    'decay':2,
    'amplitude':10e-1,
    'disable_after':60,
    'update_env':0
}

In [51]:
for l in psis:
    for psi in l:
        psi.canonical_form()

In [52]:
outpsis = list()
energies = list()

for l1, l2 in zip(psis, models):
    outpsis.append(list())
    energies.append(list())

        
    for psi, m in zip(l1, l2):
        try:
            eng = dmrg.TwoSiteDMRGEngine(psi, m, dmrg_params)
            e, psi_out = eng.run()
        except:
            e, psi_out = None, None

        outpsis[-1].append(psi_out)
        energies[-1].append(e)

/Users/kierancooney/.pyenv/versions/num_spt_venv_p11/lib/python3.11/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for config TwoSiteDMRGEngine:
['amplitude', 'decay', 'disable_after', 'update_env']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))
/Users/kierancooney/.pyenv/versions/num_spt_venv_p11/lib/python3.11/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for config TwoSiteDMRGEngine:
['amplitude', 'decay', 'disable_after', 'update_env']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))
/Users/kierancooney/.pyenv/versions/num_spt_venv_p11/lib/python3.11/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for config TwoSiteDMRGEngine:
['amplitude', 'decay', 'disable_after', 'update_env']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))
/Users/kierancooney/.pyenv/versions/num_spt_venv_p11/lib/python3.11/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for c

In [55]:
energies

[[-96.0000000000011,
  -61.932424627532924,
  -61.93242462752965,
  -61.9324246248189],
 [-96.00000000000038,
  -61.93197511895478,
  -61.93197511895475,
  -61.93197511895518]]

In [56]:
len(outpsis[0][0].sites)

100

In [57]:
outpsis[0][0].get_SR(10)

array([0.5, 0.5, 0.5, 0.5])

In [58]:
outpsis[0][0].get_SR(9)

array([0.5, 0.5, 0.5, 0.5])

In [59]:
outpsis[0][0].get_SR(11)

array([0.5, 0.5, 0.5, 0.5])

In [60]:
boson_expectations = np.zeros((2, 2, 4, 50))

indices = product(
    enumerate(['X', 'Z']),
    range(2),
    range(4)
)

for (i, op), j, k in indices:
    try:
        psi = outpsis[j][k]
        
        data = (
            psi
            .expectation_value(
                op,
                list(range(0, 100, 2))
            )
        )

        boson_expectations[i,j,k] = data

    except:
        boson_expectations[i,j,k] = np.nan

/var/folders/r3/xn8xq5c17932m2g1b4r3dss80000gn/T/ipykernel_79511/3510999881.py:21: ComplexWarning: Casting complex values to real discards the imaginary part
  boson_expectations[i,j,k] = data


In [61]:
boson_expectations[0]

array([[[ 0.00000000e+00,  3.27423003e-14, -3.82333054e-15,
          9.67975700e-16,  3.52495810e-15, -2.99760217e-15,
         -4.64905892e-16,  1.27328703e-15,  5.23886490e-16,
         -1.38777878e-16, -3.27342320e-15, -1.81452076e-15,
          6.01949046e-16,  4.71324368e-15,  3.05658276e-15,
          2.49106291e-15,  4.37150316e-16, -8.32667268e-17,
          3.92047506e-16, -2.27248775e-16, -2.87964097e-16,
         -7.91033905e-16, -4.87804241e-15,  1.96370697e-15,
          1.40339129e-15,  9.32587341e-15, -1.82839854e-15,
          4.82253126e-16, -4.33680869e-17,  1.17614252e-15,
         -3.50414142e-16,  3.46944695e-17, -2.42861287e-16,
         -2.64111649e-16,  2.39391840e-16,  1.68268177e-16,
          8.43075609e-16, -4.93875774e-15, -6.34908792e-16,
         -1.90125693e-15,  6.52256027e-16,  1.31492039e-15,
         -7.73686670e-16, -2.01227923e-16,  4.54497551e-16,
         -3.70536934e-15,  3.13074219e-15, -4.59701721e-16,
         -9.80456427e-14,  0.00000000e+0

In [62]:
boson_expectations[1]

array([[[ 1.00000000e+00, -1.58276170e-14,  3.45209972e-15,
          7.49400542e-15, -1.21248497e-14,  5.05151476e-15,
          2.67147415e-14,  1.16295862e-14,  1.29618538e-14,
         -2.99760217e-15, -2.56461519e-14, -2.51187959e-14,
         -1.98400324e-14, -2.00395256e-14, -1.15879528e-14,
          2.13162821e-14,  4.13558077e-15,  1.01862963e-14,
          1.19348975e-14, -2.40085729e-14, -1.90125693e-14,
         -5.06539255e-15,  3.28209682e-15, -6.35602682e-15,
          2.60902411e-15,  8.50708393e-15, -1.30347122e-14,
          8.18789481e-16, -1.86586857e-14, -2.50077736e-14,
         -2.41473508e-14,  1.47104551e-14, -3.10446113e-14,
          9.10382880e-15, -4.44089210e-15, -1.66602843e-14,
          2.64371858e-15,  2.87547763e-14, -3.11417558e-14,
         -1.54147528e-14, -5.34294831e-15, -6.20337115e-15,
          4.39370762e-14,  3.55826479e-14, -1.60635394e-14,
         -1.75554016e-14, -2.12295459e-14, -2.06050454e-14,
          9.85322934e-16,  1.00000000e+0

In [63]:
fermion_number_expectations = np.zeros((2, 4, 50))

indices = product(
    range(2),
    range(4)
)

for j, k in indices:
    try:
        psi = outpsis[j][k]
        
        data = (
            psi
            .expectation_value(
                'N',
                list(range(1, 100, 2))
            )
        )

        fermion_number_expectations[j,k] = data

    except:
        fermion_number_expectations[j,k] = np.nan

In [64]:
fermion_number_expectations

array([[[0.00000000e+00, 3.60894352e-58, 1.00000000e+00, 1.78097057e-32,
         2.36245007e-32, 1.61270528e-31, 1.00000000e+00, 1.97320846e-30,
         5.95719063e-32, 1.00000000e+00, 1.19128999e-31, 3.31206610e-32,
         1.00000000e+00, 1.86305473e-31, 6.34328111e-32, 1.00000000e+00,
         1.00000000e+00, 5.12253294e-31, 4.32462808e-31, 1.00000000e+00,
         6.44220246e-32, 1.00000000e+00, 1.08519276e-31, 7.68449018e-32,
         8.95483246e-32, 1.00000000e+00, 1.00000000e+00, 1.42298896e-32,
         2.57643569e-31, 1.00000000e+00, 7.16436652e-32, 2.61000402e-31,
         1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 2.75726001e-31,
         1.73381339e-31, 4.17542795e-32, 1.00000000e+00, 1.00000000e+00,
         6.01670806e-31, 4.40795498e-32, 1.00000000e+00, 9.37748994e-32,
         1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
         1.00000000e+00, 0.00000000e+00],
        [8.54226110e-01, 8.02790632e-01, 8.55135412e-01, 8.47515568e-01,
         

In [65]:
boson_correlations = np.zeros((2, 2, 4, 50, 50))

indices = product(
    enumerate(['X', 'Z']),
    range(2),
    range(4)
)

for (i, op), j, k in indices:
    try:
        psi = outpsis[j][k]
        
        data = (
            psi
            .correlation_function(
                op,
                op,
                list(range(0, 100, 2)),
                list(range(0, 100, 2)),
            )
        )
    
        boson_correlations[i,j,k] = data

    except:
        boson_correlations[i,j,k] = np.nan

/var/folders/r3/xn8xq5c17932m2g1b4r3dss80000gn/T/ipykernel_79511/290460682.py:23: ComplexWarning: Casting complex values to real discards the imaginary part
  boson_correlations[i,j,k] = data


In [66]:
boson_correlations[..., 1, -2]

array([[[-3.29346245e-29,  1.82821221e-01, -7.90718694e-33,
          1.82821225e-01],
        [ 1.00000000e+00,  2.50000000e-01, -1.93208859e-29,
          2.50000000e-01]],

       [[-1.37582272e-28, -5.30020610e-15,  9.97884813e-31,
          7.40397560e-16],
        [-1.44267170e-28,  1.82519582e-15, -1.84405897e-30,
          6.81491031e-15]]])

In [67]:
fermion_correlations = np.zeros((2, 4, 50, 50))

indices = product(
    range(2),
    range(4)
)

for j, k in indices:
    try:
        psi = outpsis[j][k]
        
        data = (
            psi
            .correlation_function(
                'N',
                'N',
                list(range(1, 101, 2)),
                list(range(1, 101, 2)),
            )
        )
    
        fermion_correlations[j,k] = data

    except:
        fermion_correlations[j,k] = np.nan

In [68]:
fermion_correlations[..., 1, -2]

array([[3.60894352e-58, 6.85764712e-01, 6.85764712e-01, 6.85764718e-01],
       [2.59864839e-01, 6.85826082e-01, 6.85826082e-01, 6.85826082e-01]])

In [69]:
reduced_boson_correlations = (
    boson_correlations -
    boson_expectations[...,np.newaxis]*boson_expectations[...,np.newaxis,:]
)

In [71]:
reduced_boson_correlations.shape

(2, 2, 4, 50, 50)

In [72]:
reduced_boson_correlations[..., 25, 26]

array([[[ 3.39695524e-28, -3.09943994e-03,  6.44528116e-31,
         -3.09941313e-03],
        [ 1.11022302e-16,  2.77555756e-17, -3.47778551e-30,
         -5.55111512e-17]],

       [[-1.30847495e-14, -1.76347875e-01,  4.97104305e-18,
         -1.76347796e-01],
        [ 3.76361326e-29, -1.76393689e-01, -5.98757088e-30,
         -1.76393689e-01]]])

In [73]:
reduced_fermion_correlations = (
    fermion_correlations -
    fermion_number_expectations[...,np.newaxis]*fermion_number_expectations[...,np.newaxis,:]
)

In [74]:
reduced_fermion_correlations[..., 25, 26]

array([[ 0.00000000e+00,  5.58644098e-02,  5.58644098e-02,
         5.58516000e-02],
       [-1.08420217e-19,  5.58383349e-02,  5.58383349e-02,
         5.58383349e-02]])

## Correct possible factor issue... again.

In [75]:
def get_op_list_1(g_quad, n1_func):
    out = list()
    g_left, g_in, g_out, g_right = g_quad

    out.append((f'map_{g_left}_{g_left}', 0, 0))

    if n1_func(g_in, g_right):
        out.append(('Cd', 1, 1))
    if n1_func(g_left, g_in):
        out.append(('Cd', 0, 1))
    
    out.append((f'map_{g_in}_{g_out}', 1, 0))
    
    if n1_func(g_left, g_out):
        out.append(('C', 0, 1))
    if n1_func(g_out, g_right):
        out.append(('C', 1, 1))

    out.append((f'map_{g_right}_{g_right}', 2, 0))

    return out

In [76]:
class ClusterFermion1(CouplingMPOModel):
        default_lattice = "Chain"
        force_default_lattice = True

        # Would it be easier to subclass ClockSite?
        def init_sites(self, model_params):
            spin = ClockSite(4, conserve=None)
            for i in range(4):
                for j in range(4):
                    X = np.zeros((4,4))
                    X[j, i] = 1
                    op_name = f"map_{i}_{j}"
                    spin.add_op(op_name, X)
            ferm = FermionSite(conserve=None)
            sites = [spin, ferm]
            return [spin, ferm], ['s', 'f']

        def init_terms(self, model_params):
            # Read off model parameters
            n1_01, n1_10 = model_params.get('n1', (0,0))
            n1_func = get_n1_func(n1_01, n1_10)

            non_trivial_proj_rep = model_params.get('non_trivial_proj_rep', False)

            group_quads = product(range(4), repeat=4)

            for group_quad in group_quads:
                op_list = get_op_list_1(group_quad, n1_func)
                
                if non_trivial_proj_rep:
                    phase = get_proj_rep_term_phase(group_quad)
                else:
                    phase = 1

                g_left, *_, g_right = group_quad
                factor=1

                self.add_multi_coupling((-1/4)*phase*factor, op_list)

In [77]:
models = [
    [ClusterFermion1({'L': 50, 'n1': p, 'non_trivial_proj_rep': b}) for p in n1_pairs]
    for b in [True, False]
]

In [78]:
models

[[<__main__.ClusterFermion1 at 0x317996cd0>,

In [79]:
psis = [
    [MPS.from_lat_product_state(m.lat, [['0', 'empty'],]*50) for m in l]
    for l in models
]

In [80]:
dmrg_params = {
    "trunc_params": {"chi_max": 8, "chi_min": 1, "svd_min": 1.e-10},
    "min_sweeps":100,
    "max_sweeps":200,
    "mixer": True,
    "combine":False,
    'decay':2,
    'amplitude':10e-1,
    'disable_after':60,
    'update_env':0
}

In [81]:
for l in psis:
    for psi in l:
        psi.canonical_form()

In [82]:
outpsis = list()
energies = list()

for l1, l2 in zip(psis, models):
    outpsis.append(list())
    energies.append(list())

        
    for psi, m in zip(l1, l2):
        try:
            eng = dmrg.TwoSiteDMRGEngine(psi, m, dmrg_params)
            e, psi_out = eng.run()
        except:
            e, psi_out = None, None

        outpsis[-1].append(psi_out)
        energies[-1].append(e)

/Users/kierancooney/.pyenv/versions/num_spt_venv_p11/lib/python3.11/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for config TwoSiteDMRGEngine:
['amplitude', 'decay', 'disable_after', 'update_env']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))
/Users/kierancooney/.pyenv/versions/num_spt_venv_p11/lib/python3.11/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for config TwoSiteDMRGEngine:
['amplitude', 'decay', 'disable_after', 'update_env']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))
/Users/kierancooney/.pyenv/versions/num_spt_venv_p11/lib/python3.11/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for config TwoSiteDMRGEngine:
['amplitude', 'decay', 'disable_after', 'update_env']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))
/Users/kierancooney/.pyenv/versions/num_spt_venv_p11/lib/python3.11/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for c

In [83]:
energies

[[-48.0000000000005,
  -32.81197648970377,
  -32.81197648970407,
  -32.81197648970407],
 [-48.000000000000085,
  -32.81137784106575,
  -32.81102779003888,
  -32.81120103606945]]

In [84]:
len(outpsis[0][0].sites)

100

In [85]:
outpsis[0][0].get_SR(10)

array([0.5, 0.5, 0.5, 0.5])

In [86]:
outpsis[0][0].get_SR(9)

array([0.5, 0.5, 0.5, 0.5])

In [87]:
outpsis[0][0].get_SR(11)

array([0.5, 0.5, 0.5, 0.5])

In [88]:
boson_expectations = np.zeros((2, 2, 4, 50))

indices = product(
    enumerate(['X', 'Z']),
    range(2),
    range(4)
)

for (i, op), j, k in indices:
    try:
        psi = outpsis[j][k]
        
        data = (
            psi
            .expectation_value(
                op,
                list(range(0, 100, 2))
            )
        )

        boson_expectations[i,j,k] = data

    except:
        boson_expectations[i,j,k] = np.nan

/var/folders/r3/xn8xq5c17932m2g1b4r3dss80000gn/T/ipykernel_79511/3510999881.py:21: ComplexWarning: Casting complex values to real discards the imaginary part
  boson_expectations[i,j,k] = data


In [89]:
boson_expectations[0]

array([[[ 0.00000000e+00, -7.79567137e-15, -9.85322934e-16,
         -1.41119755e-15, -5.18422111e-15, -2.42514342e-15,
         -2.40953091e-15,  2.10248485e-15, -8.39606162e-16,
         -8.63892291e-16,  6.50521303e-16,  1.45716772e-16,
          6.31439345e-16, -4.37150316e-16,  6.01949046e-16,
         -2.15626128e-15,  5.10008702e-16,  2.01227923e-15,
         -6.89986263e-16,  1.56125113e-15, -1.28543010e-15,
         -2.28983499e-15,  8.74300632e-16, -4.08006962e-15,
          6.66133815e-16,  3.46944695e-18,  5.69683190e-15,
          5.16947596e-16, -3.98986399e-15,  2.30718222e-16,
          1.58900670e-15,  1.19695920e-15, -6.93889390e-18,
         -8.84708973e-16,  1.87350135e-15,  5.55111512e-17,
          2.57172755e-15, -2.41473508e-15, -1.18655086e-15,
          1.28716482e-15, -2.77555756e-16, -6.24500451e-17,
         -5.55111512e-16,  7.18175519e-16, -9.88792381e-16,
         -1.08246745e-15, -9.43689571e-16, -5.20417043e-16,
          3.03541030e-14,  0.00000000e+0

In [90]:
boson_expectations[1]

array([[[ 1.00000000e+00, -9.21485110e-15, -2.15938378e-14,
         -2.40224507e-14, -1.50157664e-14, -9.60342916e-15,
          2.00256478e-14,  3.32234240e-14,  1.82631688e-14,
         -1.74027459e-14,  1.10744747e-14, -7.75768338e-15,
         -5.38458167e-14, -6.22418783e-15,  2.35922393e-14,
         -2.37032616e-14,  6.68909372e-15, -7.83401122e-15,
         -4.28129754e-14, -2.85882429e-14, -3.16968674e-14,
         -2.16215934e-14, -1.14561138e-14, -2.34881559e-14,
         -2.47024623e-15,  2.24473218e-14, -2.91433544e-15,
         -6.06459327e-15,  1.51476054e-14,  1.38777878e-16,
          4.65599781e-14, -6.58501031e-15, -1.79023463e-14,
         -2.52853294e-14, -2.05668815e-14, -5.14518983e-15,
         -1.88321581e-14,  2.85882429e-15, -2.75890422e-14,
          7.27196081e-15,  1.70974346e-14,  2.03101425e-14,
         -2.74780199e-15, -4.05231404e-15, -2.43832732e-14,
         -1.29063427e-15,  1.27675648e-15, -1.68268177e-15,
         -2.06779038e-14,  1.00000000e+0

In [91]:
fermion_number_expectations = np.zeros((2, 4, 50))

indices = product(
    range(2),
    range(4)
)

for j, k in indices:
    try:
        psi = outpsis[j][k]
        
        data = (
            psi
            .expectation_value(
                'N',
                list(range(1, 100, 2))
            )
        )

        fermion_number_expectations[j,k] = data

    except:
        fermion_number_expectations[j,k] = np.nan

In [92]:
fermion_number_expectations

array([[[1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 2.27256093e-29,
         3.36704143e-31, 5.30820785e-32, 6.21023021e-32, 1.00000000e+00,
         1.00000000e+00, 2.67174290e-28, 1.00000000e+00, 1.00000000e+00,
         1.00000000e+00, 4.73666330e-31, 1.88013136e-31, 2.67383690e-29,
         1.00000000e+00, 1.00000000e+00, 3.66745346e-32, 8.79119808e-32,
         8.07381653e-32, 7.10372603e-32, 1.00000000e+00, 1.00000000e+00,
         2.09629089e-32, 1.57050773e-31, 1.55143935e-31, 1.28249743e-31,
         1.00000000e+00, 1.00000000e+00, 2.02159253e-31, 1.00000000e+00,
         7.46080879e-32, 1.86407726e-27, 1.00000000e+00, 3.40935908e-32,
         1.00000000e+00, 1.00000000e+00, 2.74415578e-32, 2.31492548e-32,
         1.00000000e+00, 8.05597885e-32, 1.19821484e-31, 7.85590297e-32,
         1.00000000e+00, 3.16869088e-31, 1.00000000e+00, 4.09195291e-32,
         1.00000000e+00, 0.00000000e+00],
        [7.42737034e-01, 7.17357746e-01, 7.29685479e-01, 7.23643618e-01,
         

In [93]:
boson_correlations = np.zeros((2, 2, 4, 50, 50))

indices = product(
    enumerate(['X', 'Z']),
    range(2),
    range(4)
)

for (i, op), j, k in indices:
    try:
        psi = outpsis[j][k]
        
        data = (
            psi
            .correlation_function(
                op,
                op,
                list(range(0, 100, 2)),
                list(range(0, 100, 2)),
            )
        )
    
        boson_correlations[i,j,k] = data

    except:
        boson_correlations[i,j,k] = np.nan

/var/folders/r3/xn8xq5c17932m2g1b4r3dss80000gn/T/ipykernel_79511/290460682.py:23: ComplexWarning: Casting complex values to real discards the imaginary part
  boson_correlations[i,j,k] = data


In [94]:
boson_correlations[..., 1, -2]

array([[[ 1.20751113e-28,  7.26908302e-02,  3.63500168e-33,
          7.26908302e-02],
        [ 1.00000000e+00,  2.50000000e-01, -5.10342414e-29,
          2.50000000e-01]],

       [[ 3.68792473e-29,  2.93898779e-15,  4.14791678e-31,
         -1.66605902e-15],
        [-2.01883663e-28,  1.67571169e-15,  5.01851505e-30,
         -3.66658800e-15]]])

In [95]:
fermion_correlations = np.zeros((2, 4, 50, 50))

indices = product(
    range(2),
    range(4)
)

for j, k in indices:
    try:
        psi = outpsis[j][k]
        
        data = (
            psi
            .correlation_function(
                'N',
                'N',
                list(range(1, 101, 2)),
                list(range(1, 101, 2)),
            )
        )
    
        fermion_correlations[j,k] = data

    except:
        fermion_correlations[j,k] = np.nan

In [96]:
fermion_correlations[..., 1, -2]

array([[1.        , 0.53280886, 0.53280886, 0.53280886],
       [0.11595859, 0.53293279, 0.53309587, 0.53298606]])

In [97]:
reduced_boson_correlations = (
    boson_correlations -
    boson_expectations[...,np.newaxis]*boson_expectations[...,np.newaxis,:]
)

In [98]:
reduced_boson_correlations.shape

(2, 2, 4, 50, 50)

In [99]:
reduced_boson_correlations[..., 25, 26]

array([[[ 1.82719474e-29,  2.26480709e-02, -3.97821174e-29,
          2.26480709e-02],
        [ 0.00000000e+00, -1.11022302e-16, -6.09617871e-28,
          1.11022302e-16]],

       [[-5.93590187e-17, -2.29599338e-03,  1.33477864e-18,
         -2.29599338e-03],
        [ 5.62799112e-29, -2.33753424e-03, -9.01436657e-30,
         -2.35307168e-03]]])

In [100]:
reduced_fermion_correlations = (
    fermion_correlations -
    fermion_number_expectations[...,np.newaxis]*fermion_number_expectations[...,np.newaxis,:]
)

In [101]:
reduced_fermion_correlations[..., 25, 26]

array([[-3.46007299e-63,  4.14324189e-02,  4.14324189e-02,
         4.14324189e-02],
       [-3.46944695e-18,  4.13937360e-02,  4.14197007e-02,
         4.12223635e-02]])